In [8]:
import numpy as np
from copy import deepcopy
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

In [5]:
def show_board(board):
    output = ''
    first_line = True
    for line in board:
        first_element = True
        if first_line:
            for element in line:
                if first_element:
                    first_element = False
                    output += f'{element}'   
                else:
                    output += f'|{element}'          
            first_line = False
        else:
            output += '\n-----\n'
            for element in line:
                if first_element:
                    first_element = False
                    output += f'{element}'   
                else:
                    output += f'|{element}'
    return output

In [6]:
def row_equal(row):
    if len(set(row)) == 1 and ' ' not in row:
        return True

In [4]:
def check_winner(board):
    for row in board:
        if row_equal(row):
            return row[0]
    for column in board.T:
        if row_equal(column):
            return column[0] 
    if row_equal(np.diag(board)):
        return np.diag(board)[0]
    if row_equal(np.diag(np.fliplr(board))):
        return np.diag(np.fliplr(board))[0]
    return None

In [3]:
def available_moves(board):
    return np.argwhere(board == ' ')

## Vanilla Minimax

In [21]:
reg_nodes = 0

In [20]:
def minimax(board, depth, state, quiet = False):
    global reg_nodes
    reg_nodes += 1
    tab = depth*'\t'
    if not quiet:
        print(tab + show_board(board).replace('\n', f'\n{tab}'))
    if check_winner(board):
        if check_winner(board) == 'X':
            return 1
        else:
            return -1
    elif len(available_moves(board)) == 0:
        return 0
    if state == 'X':
        value = -100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = max(value, minimax(board_copy, depth + 1, 'O', quiet = quiet))
        return value
    elif state == 'O':
        value = 100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = min(value, minimax(board_copy, depth + 1, 'X', quiet = quiet))
        return value

## Minimax with alpha beta pruning

In [34]:
alpha_beta_nodes = 0

In [35]:
def alpha_beta_minimax(board, depth, state, alpha, beta, quiet = False):
    '''
    On first call set alpha = -100 and beta = 100
    '''
    global alpha_beta_nodes
    alpha_beta_nodes += 1
    tab = depth*'\t'
    if not quiet:
        print(tab + show_board(board).replace('\n', f'\n{tab}'))
    if check_winner(board):
        if check_winner(board) == 'X':
            return 1
        else:
            return -1
    elif len(available_moves(board)) == 0:
        return 0
    if state == 'X':
        value = -100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = max(value, alpha_beta_minimax(board_copy, depth + 1, 'O', alpha, beta, quiet = quiet))
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value
    elif state == 'O':
        value = 100
        for move in available_moves(board):
            board_copy = deepcopy(board)
            board_copy[move[0],move[1]] = state
            value = min(value, alpha_beta_minimax(board_copy, depth + 1, 'X', alpha, beta, quiet = quiet))
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

## Comparing the number of iterations

In [16]:
board = np.full((3, 3), ' ')

In [17]:
board[0,0] = 'X'

In [23]:
minimax(board, 0, 'O', quiet = True)

0

In [36]:
alpha_beta_minimax(board, 0, 'O', -100, 100, quiet=True)

0

In [40]:
print(f'Number of iterations of regular implementation: {reg_nodes}')
print(f'Number of iterations with alpha beta pruning: {alpha_beta_nodes}')

Number of iterations of regular implementation: 59705
Number of iterations with alpha beta pruning: 2338


## Cool it looks like our alpha beta pruning algorithm improves computation quite a bit